In [1]:
import csv
import snscrape.modules.twitter as sntwitter
import itertools
import pandas as pd
import json

In [2]:
pd.options.display.max_columns = None

In [3]:
OUTPUT_FILE = "../data/results.csv"

In [4]:
def save_tweets(query, n_tweets):
    """
    Finds the latest tweets according to a query and saves them in a csv file
    :param query: the query to look for
    :param n_tweets: number of tweets to scrape
    """
    with open(OUTPUT_FILE, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['id', 'date', 'username', 'content', 'url'])
        
        tweets_gen = sntwitter.TwitterSearchScraper(query).get_items()
        top_tweets = itertools.islice(tweets_gen, n_tweets)
        
        for tweet in top_tweets:
            writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])
        
        print('Done!')

In [5]:
query = "#foreignaid"

In [6]:
save_tweets(query=query, n_tweets=10)

Done!


/tmp/ipykernel_12009/2365005399.py:15: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])
/tmp/ipykernel_12009/2365005399.py:15: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  writer.writerow([tweet.id, tweet.date, tweet.username, tweet.content, tweet.url])


In [7]:
tweets_gen = sntwitter.TwitterSearchScraper(query).get_items()

## Advanced

Advanced Search reference: https://github.com/igorbrigadir/twitter-advanced-search

In [8]:
def advancedTweetSearch(query, loc, start, end, lang="en"):
    TWEET_STOR = []
    LOC_STOR = []
    
    scraped_tweets = sntwitter.TwitterSearchScraper(f'{query} geocode:"{loc}"since:{start} until:{end}').get_items()
    
    sliced_scraped_tweets = itertools.islice(scraped_tweets, 10000)
    
    
    for tweet in sliced_scraped_tweets:
        try:
            LOC_STOR.append(json.loads(tweet.json())['place']['country'])
            TWEET_STOR.append(json.loads(tweet.json()))
        except TypeError:
            continue
        
    return TWEET_STOR, LOC_STOR

In [9]:
query = "economic development OR #economicdevelopment"
start = '2019-01-01'
end = '2023-01-01'
loc = "-3.337954,119.882813, 4000km"

In [10]:
%time southest_asia, location_details = advancedTweetSearch(query=query, start=start, end=end, loc=loc)

CPU times: user 11 s, sys: 132 ms, total: 11.1 s
Wall time: 6min 17s


In [11]:
pd.Series(location_details).value_counts()

India                         3066
Pakistan                       453
Australia                      311
Bangladesh                     113
Maldives                        98
Sri Lanka                       72
Nepal                           61
People's Republic of China      47
Indonesia                       27
Taiwan                          20
East Timor                      10
Papua New Guinea                 7
Bhutan                           6
Northern Mariana Islands         3
Guam                             3
Japan                            3
                                 2
United States                    1
dtype: int64

In [12]:
set([(tweet['lang']) for tweet in southest_asia])

{'ca',
 'da',
 'en',
 'es',
 'et',
 'fr',
 'hi',
 'ht',
 'in',
 'mr',
 'nl',
 'pt',
 'ro',
 'tl',
 'und',
 'zh'}

In [13]:
# brazil

loc = "-8.581021,-51.328125, 5000km"

In [14]:
%time lat_amrica, location_details = advancedTweetSearch(query=query, start=start, end=end, loc=loc)

CPU times: user 14.4 s, sys: 144 ms, total: 14.5 s
Wall time: 7min 47s


In [15]:
pd.Series(location_details).value_counts()

United States    4302
Ghana             496
Mexico             58
Sierra Leone       46
Morocco            31
Gambia             25
Fiji               24
Spain              19
Ivory Coast        19
Senegal            18
Brazil             16
Liberia            15
Bahamas            10
Guatemala           7
Bermuda             6
Cape Verde          4
Guinea              3
Mali                2
Portugal            1
Gibraltar           1
Burkina Faso        1
Ecuador             1
Argentina           1
Mauritania          1
Guinea Bissau       1
dtype: int64

In [16]:
set([tweet['lang'] for tweet in lat_amrica])

{'ca', 'en', 'es', 'fr', 'pt', 'ro'}

# store tweets

In [3]:
import pickle

In [6]:
with open('../data/01_raw_latin_amr_tweets.pickle', 'rb') as f:
    lat_amrica = pickle.load(f)
f.close()

In [7]:
with open('../data/01_raw_southest_asia_tweets.pickle', 'rb') as f:
    southest_asia = pickle.load(f)
f.close()

In [43]:
# with open('../data/01_raw_latin_amr_tweets.pickle', 'wb') as f:
#     pickle.dump(lat_amrica, f)
# f.close()

In [45]:
# with open('../data/01_raw_southest_asia_tweets.pickle', 'wb') as f:
#     pickle.dump(southest_asia, f)
# f.close()

In [8]:
lat_amrica[0].keys()

dict_keys(['_type', 'url', 'date', 'rawContent', 'renderedContent', 'id', 'user', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'links', 'media', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates', 'place', 'hashtags', 'cashtags', 'card', 'viewCount', 'vibe', 'content', 'outlinks', 'outlinksss', 'tcooutlinks', 'tcooutlinksss', 'username'])

In [71]:
MAIN_COLS = ['url', 'date', 'rawContent', 'renderedContent', 'id', 'replyCount', 'retweetCount',
             'likeCount', 'quoteCount', 'lang', 'retweetedTweet', 'quotedTweet', 'inReplyToTweetId',
             'inReplyToUser', 'mentionedUsers', 'hashtags', 'cashtags', 
            'viewCount', 'vibe', 'content']
USER_COLS = ['username', 'id', 'rawDescription', 'renderedDescription', 'created', 'followersCount', 'statusesCount', 
            'favouritesCount', 'listedCount']
CARD_COLS = ['title', 'description', 'siteUser']

SITE_USER_COLS = ['id', 'displayname', 'rawDescription', 'renderedDescription', 'verified', 'created', 
 'followersCount', 'friendsCount', 'statusesCount', 'favouritesCount', 'listedCount', 'mediaCount', 'location']

In [72]:
lat_amr_df = pd.json_normalize(lat_amrica)

In [73]:
lat_amr_df.columns = lat_amr_df.columns.str.replace('.', '_')

/tmp/ipykernel_4686/2142218018.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lat_amr_df.columns = lat_amr_df.columns.str.replace('.', '_')


In [74]:
['user_' + col for col in USER_COLS if 'user_' + col not in lat_amr_df.columns]

[]

In [75]:
['card_' + col for col in CARD_COLS if 'card_' + col not in lat_amr_df.columns]

[]

In [76]:
['card_siteUser__' + col for col in SITE_USER_COLS if 'card_siteUser__' + col not in lat_amr_df.columns]

['card_siteUser__id',
 'card_siteUser__displayname',
 'card_siteUser__rawDescription',
 'card_siteUser__renderedDescription',
 'card_siteUser__verified',
 'card_siteUser__created',
 'card_siteUser__followersCount',
 'card_siteUser__friendsCount',
 'card_siteUser__statusesCount',
 'card_siteUser__favouritesCount',
 'card_siteUser__listedCount',
 'card_siteUser__mediaCount',
 'card_siteUser__location']

In [77]:
[col for col in lat_amr_df.columns if col.startswith("place")]

['place__type',
 'place_id',
 'place_fullName',
 'place_name',
 'place_type',
 'place_country',
 'place_countryCode']

In [78]:
all_cols = MAIN_COLS + ['user_' + col for col in USER_COLS] + \
['card_' + col for col in CARD_COLS] + ['card_siteUser_' + col for col in SITE_USER_COLS] +  ['place__type',
 'place_id',
 'place_fullName',
 'place_name',
 'place_type',
 'place_country',
 'place_countryCode']

In [94]:
lat_amr_df[all_cols]['mentionedUsers'].sample(5)

4312                                                 None
4670                                                 None
1706    [{'_type': 'snscrape.modules.twitter.User', 'u...
179                                                  None
1230                                                 None
Name: mentionedUsers, dtype: object

In [81]:
sthest_as_df = pd.json_normalize(southest_asia)

In [82]:
sthest_as_df.columns = sthest_as_df.columns.str.replace('.', '_')

/tmp/ipykernel_4686/175861509.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sthest_as_df.columns = sthest_as_df.columns.str.replace('.', '_')
